## - 개요

화장품 전성분 데이터를 수집하기 위한 본작업.
<br>
INCI-Decoder 사이트 내에서 수집한 모든 제품의 리스트를 통해
<br>
개별 url을 통해 각 페이지에 접근,
<br>
화장품에 담긴 모든 성분을 각 화장품과 매치한 테이블과
<br>
각 성분과 그에 대한 효능을 매치한 테이블을 만들 것.
<br>
즉, 화장품명, 전성분표 및 성분에 따른 효능 데이터
<br>
모두를 수집할 예정.
<br>
<br>
<br>
(이 과정에서 시간적 비용 및 리소스 낭비를 방지하기 위해,
<br>
샘플 데이터를 1000개로 제한하여 앞으로의 실습을 진행할 것)

### 1. 활용 모듈 및 성분 파일 불러오기

프로젝트에서 활용할 모듈은 사용한 셀에서 설명할 예정

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import random

import tqdm
from bs4 import BeautifulSoup
import requests
from html_table_parser import parser_functions

앞서 INCI-Decoder 사이트에서 수집한 제품명 리스트 불러오기.
<br>
해당 파일의 'product_label' 칼럼을 활용하여
<br>
INCI-Decoder 사이트 내 제품 전반에 대한 데이터에 접근할 것.

In [2]:
# INCI_Decoder의 화장품 리스트

product_lst = list(pd.read_csv("./product_df.csv")['product_label'])
len(product_lst)

77001

### 2. 데이터 수집을 위한 사전 처리

크롤링할 데이터를 선택하는 과정.
<br>
본격적인 데이터 수집에 앞서서 샘플을 추출하고,
<br>
수집할 데이터를 저장할 리스트를 구성.

In [3]:
# 1000개의 샘플 데이터를 뽑아오기 위한 사전 작업

sample_list = random.sample(product_lst, 1000)

In [4]:
product_name = [] # 화장품 이름
ingredient_lst = [] # 화장품에 들어있는 성분을 리스트로 받음
formatted_ingredient_lst = [] # formatted 성분 표기명을 리스트로 받음
what_lst = [] # 성분이 어떤 효능이 있는지 리스트로 받음
failed_lst = [] # 크롤링 중 실패한 로그를 추적하기 위해

### 3. 데이터 수집

화장품 이름, 화장품의 성분 리스트, 표준화된 성분 표기명, 성분의 효능을
<br>
화장품 이름이 포함된 url을 통해 접근한 웹페이지를 돌며 수집할 예정.
<br>
이후, 사용 목적에 맞는 두 개의 테이블을 구성하기 위해서
<br>
데이터프레임에 적합한 리스트를 넣는 작업을 병행.

In [6]:
for product in tqdm.tqdm_notebook(sample_list):
    url = f"https://incidecoder.com/products/{product}" # url에 product 명을 바꿔가며 루프를 돈다
    response = requests.get(url)

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

    data = soup.find('table',{"class" : "product-skim fs16"}) # class가 product-skim fs16인 테이블을 태그로 받음 
    df = parser_functions.make2d(data)[1:] # make2d를 활용해 테이블을 파이썬 자료구조로 받고
    tmpdf = pd.DataFrame(df, columns=["Ingredient name", "what-it-does", "irr., com.", "ID-Rating"]) # 임시 데이터 프레임을 만든다
    
    try: # 크롤링 시도
        # 클래스 bold인 성분들은 html 태그에 formatted_name이 존재하지 않았다.
        # 그래서 테이블에는 기록되어 있지만, 데이터를 모두 수집한 후 데이터 프레임으로 만드는 과정에서 개수가 맞지 않는 이슈가 있었다.(Ingredient_name != Formatted_name)
        # 이를 방지하기 위해 bold처리된 성분들을 없앨 것.
        Bold_lst = []
        for ing in data.find_all('td', {'class': 'bold'}):
            Bold_lst.append(ing.text.replace('\n','').strip()) # 전처리
        
        indexes = []
        for stop in Bold_lst: # 볼드 처리된 성분들을 없애기 위해 인덱스를 알아내는 작업
            indexes.append(tmpdf[tmpdf['Ingredient name'] == stop].index[0])
        
        tmpdf.drop(indexes, axis=0,inplace=True) # 알아온 인덱스들로 drop을 활용해 임시 데이터 프레임에서 삭제한다

        ingtmp = []
        for tag in data.find_all('a', {'class': "black ingred-detail-link"}):
            ingtmp.append(tag.attrs['href'][13:]) # html 태그에 들어있는 formatted 성분명을 알아오는 작업. 마찬가지로 리스트 형태로 받아온다
        formatted_ingredient_lst.append(ingtmp)

        # 임시로 만든 tmpdf의 데이터들을 활용해 성분명_리스트, 효능_리스트를 얻고
        ingredient_lst.append(list(tmpdf['Ingredient name']))
        what_lst.append(list(tmpdf["what-it-does"]))
        product_name.append(product) # 제품명 추가
    
    except: # 실패시 failed_lst에 기록
        failed_lst.append([product, data])

  0%|          | 0/1000 [00:00<?, ?it/s]

성분명, 표준화된 성분 표기명, 성분의 효능이 포함된
<br>
두 번째 테이블을 구성하기 위해 진행하는 부차적인 작업

In [7]:
# 리스트로 묶여있는 원소들을 개별로 받아오는 작업

each_ingredient_lst = []
for lst in ingredient_lst:
    for ing in lst:
        each_ingredient_lst.append(ing)

each_formatted_ingredient_lst = []
for lst in formatted_ingredient_lst:
    for ing in lst:
        each_formatted_ingredient_lst.append(ing)

each_what_lst = []
for lst in what_lst:
    for does in lst:
        tmp = []
        for does in does.replace('\n','').replace('\u200b','').split(','): # 전처리
            tmp.append(does.strip())
        each_what_lst.append(tmp)

수집한 데이터들이 포함된 리스트를 활용하여
<br>
두 개의 테이블을 구성.

In [12]:
# 제품명, 전성분표 리스트, 표준 성분 표기명으로 구성된 첫 번째 테이블

cols = ['product name', 'ingredients', 'formatted ingredients']
product_df = pd.DataFrame(columns=cols)

product_df['product name'] = product_name
product_df['ingredients'] = ingredient_lst
product_df['formatted ingredients'] = formatted_ingredient_lst

In [13]:
# 성분명, 표준 성분 표기명, 성분의 효능으로 구성된 두 번째 테이블

ingredient_df = pd.DataFrame(columns=['ingredients','formatted ingredients','what-it-does'])
ingredient_df['ingredients'] = each_ingredient_lst
ingredient_df['formatted ingredients'] = each_formatted_ingredient_lst
ingredient_df['what-it-does'] = each_what_lst

In [10]:
product_df.to_csv('inci_ing_sample.csv', index=False)

In [11]:
ingredient_df.to_csv('inci_prod_sample.csv', index=False)

### 4. 수집 데이터 확인

수집한 데이터를 확인하는 과정.
<br>
수집된 데이터가 이상이 없는지 확인.

In [15]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   product name           1000 non-null   object
 1   ingredients            1000 non-null   object
 2   formatted ingredients  1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [17]:
product_df

,product name,ingredients,formatted ingredients
0,barry-m-flawless-original-primer,"[Cyclopentasiloxane, Aqua, Isododecane, Cycloh...","[cyclopentasiloxane, water, isododecane, cyclo..."
1,annayake-bamboo-energizing-face-care,"[Aqua (Water), Propylheptyl Caprylate, Shea Bu...","[water, propylheptyl-caprylate, shea-butter-et..."
2,clarins-booster-repair,"[Glycerin, Aqua/Water/Eau, Propylene Glycol, P...","[glycerin, water, propylene-glycol, panthenol,..."
3,thermal-shampoo,"[Aqua (Medicinal Water), Sodium Laureth Sulfat...","[water, sodium-laureth-sulfate, cocamidopropyl..."
4,aya-natural-skin-calming-cream,"[Purified Aqua, Cetearyl Alcohol, Cetearyl Glu...","[water, cetearyl-alcohol, cetearyl-glucoside, ..."
...,...,...,...
995,nivea-sun-protect-sensitive,"[Aqua, Homosalate, Alcohol Denat, Glycerin, Bu...","[water, homosalate, alcohol-denat, glycerin, b..."
996,ocare-love-basic-hair-shampoo,"[Aqua, Sodium Lauroyl Methyl Isethionate, Sodi...","[water, sodium-lauroyl-methyl-isethionate, sod..."
997,lancome-tonique-douceur,"[Water, Glycerin, Sodium Citrate, Ci 42090/Blu...","[water, glycerin, sodium-citrate, ci-42090, ci..."
998,faces-canada-second-skin-foundation,"[Dimethicone, Cyclohexasiloxane, Isododecane, ...","[dimethicone, cyclohexasiloxane, isododecane, ..."


In [16]:
ingredient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26394 entries, 0 to 26393
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ingredients            26394 non-null  object
 1   formatted ingredients  26394 non-null  object
 2   what-it-does           26394 non-null  object
dtypes: object(3)
memory usage: 618.7+ KB


In [18]:
ingredient_df

,ingredients,formatted ingredients,what-it-does
0,Cyclopentasiloxane,cyclopentasiloxane,"[emollient, solvent]"
1,Aqua,water,[solvent]
2,Isododecane,isododecane,"[emollient, solvent]"
3,Cyclohexasiloxane,cyclohexasiloxane,"[emollient, solvent]"
4,Dimethicone Crosspolymer,dimethicone-crosspolymer,[viscosity controlling]
...,...,...,...
26389,Camellia Sinensis (Organic Green Tea) Extract,camellia-sinensis-leaf-extract,"[antioxidant, soothing]"
26390,Centella Asiatica (Organic Gotu Kola) Extract,centella-asiatica-extract,"[soothing, antioxidant, moisturizer/humectant]"
26391,Equisetum Arvense (Horsetail) Extract,equisetum-arvense-extract,"[soothing, emollient]"
26392,Geranium Maculatum (Wild Geranium) Extract,geranium-maculatum-extract,[]
